## <CENTER>  Mätning av $Z$-bosonens massa med ATLAS-data

### Börja med att hämta datan och ladda in de i ROOT-träd.

In [1]:
import ROOT

Welcome to JupyROOT 6.14/04


In [2]:
# Leta på följande webbadress, där ni bör hitta två filer, en för händelser med elektroner och en för myoner
# http://opendata.atlas.cern/release/samples
data_file_electrons = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/Data/DataEgamma.root")
data_file_electrons_simulation = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/MC/mc_147770.Zee.root")
data_file_muons = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/Data/DataMuons.root")

In [3]:
# Ladda in i träd (TFile::ls() kan användas för att lista filens innehåll)
tree_electrons = data_file_electrons.Get("mini")
tree_electrons_simulation = data_file_electrons_simulation.Get("mini")
tree_muons = data_file_muons.Get("mini")

### Funktioner för invariant massa

In [4]:
def getInvMass(lep1_pt, lep1_eta, lep1_phi, lep2_pt, lep2_eta, lep2_phi):
    ''' räkna ut invarianta massan med formeln i Introduktions-notboken (antar masslösa partiklar) '''
    import math
    msq = 2*lep1_pt*lep2_pt*(math.cosh(lep1_eta-lep2_eta) - math.cos(lep1_phi-lep2_phi))
    return math.sqrt(msq)

In [5]:
def getInvMassTL(lep1_pt, lep1_eta, lep1_phi, lep1_e, lep2_pt, lep2_eta, lep2_phi, lep2_e):
    ''' räkna ut invarianta massan med TLorentzVectors '''
    lep1=ROOT.TLorentzVector()
    lep2=ROOT.TLorentzVector()
    lep1.SetPtEtaPhiE(lep1_pt, lep1_eta, lep1_phi, lep1_e)
    lep2.SetPtEtaPhiE(lep2_pt, lep2_eta, lep2_phi, lep2_e)
    invmass=lep1+lep2
    return invmass.M()

### Förbered histogram

In [6]:
# Definiera binnar
nbins=50
lowedge=30.
upedge=150.
# histogram för elektron-händelser och myon-händelser
h_mass_electrons = ROOT.TH1F("h_mass_electrons", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)
h_mass_muons = ROOT.TH1F("h_mass_muons", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)

### Kör event-loopen, fyll histogram

In [7]:
# Loopa över alla händelser, räkna ut den invarianta massan av elektron-positron-par och myon-antimyon-par
# Detta kan t.ex. göras med hjälp av ROOT-klassen TLorentzVector eller genom att skriva en egen funktion.
# Fyll histogram med den invarianta massan.
# Använd vad ni lärde dig i första Introduktion-notebook.
# Nu tittar ni på data, så varje händelse kommer inte att innehålla en Z-boson.
# Ni måste själva sortera ut de händelser som har de karakteristiska drag man förväntar sig för Z-bosoner.

ievt = 0
for evt in tree_electrons:
    if evt.lep_n != 2: continue
    if not (evt.lep_type[0]==11 and evt.lep_type[1]==11): continue
    if (evt.lep_charge[0] + evt.lep_charge[1]) != 0: continue 
    if ievt > 4000:
        break
        
    # invmass_func =     getInvMass(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0],                    evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1])
    # invmass_tlorentz=getInvMassTL(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0], evt.lep_E[0]*1e-3, evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1], evt.lep_E[1]*1e-3)
    # print("invmass1 = {}, invmass2 = {}".format(invmass_func, invmass_tlorentz))
    h_mass_electrons.Fill(getInvMass(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0], evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1]))
    
    ievt += 1
    

In [8]:
# Gör samma sak för MC, så kan vi rita upp den med
h_mass_electrons_simulation = ROOT.TH1F("h_mass_electrons_simulation", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)
h_mass_muons_simulation = ROOT.TH1F("h_mass_muons_simulation", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)
ievt = 0
for evt in tree_electrons_simulation:
    if evt.lep_n != 2: continue
    if not (evt.lep_type[0]==11 and evt.lep_type[1]==11): continue
    if (evt.lep_charge[0] + evt.lep_charge[1]) != 0: continue 
    if ievt > 6000:
        break
        
    # invmass_func =     getInvMass(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0],                    evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1])
    # invmass_tlorentz=getInvMassTL(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0], evt.lep_E[0]*1e-3, evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1], evt.lep_E[1]*1e-3)
    # print("invmass1 = {}, invmass2 = {}".format(invmass_func, invmass_tlorentz))
    h_mass_electrons_simulation.Fill(getInvMass(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0], evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1]))
    
    ievt += 1

### Rita upp den invarianta massan

In [17]:
%jsroot on

In [18]:
canvas = ROOT.TCanvas("canvas", "", 800, 600)
# Draw data
h_mass_electrons.Draw("e1p")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas


In [ ]:
# Draw MC
h_mass_electrons_simulation.Scale(0.5)
h_mass_electrons_simulation.SetFillColor(ROOT.kRed)
h_mass_electrons_simulation.SetMarkerColor(ROOT.kRed)
h_mass_electrons_simulation.SetLineColor(ROOT.kRed)
h_mass_electrons_simulation.Draw("he1 SAME")
canvas.Draw()

### Anpassning med normalkurva

In [ ]:
# Gör en anpassning till peaken med rätt funktion
# prova först dummy Gauss
h_mass_electrons.Fit("gaus", "S", "SAME", 80., 100.)

In [ ]:
canvas.Draw()

### Normalkurvan ser inte så bra ut, vi provar mer korrekt modell

In [11]:
#### Prepare: setting Breit-Wigner limits, colors on functions etc.
norm_low = 100; norm_up = 15e3
mean_low = 80; mean_up = 100
width_low = 0.1; width_up = 40
def setBWLims(func):
    ''' Set limits on BW parameters '''
    func.SetParameters(0.5*(norm_low+norm_up), 0.5*(mean_low+mean_up), 0.5*(width_low+width_up))
    func.SetParLimits(0, norm_low, norm_up)
    func.SetParLimits(1, mean_low, mean_up)
    func.SetParLimits(2, width_low, width_up)
def setColor(ROOT_obj, color=ROOT.kBlack):
    ''' set (line) color'''
    ROOT_obj.SetLineColor(color)

In [19]:
#### DEFINE SOME BREIT-WIGNER FUNCTIONS

xlow_fit=30.; xup_fit=140.;
#foo=ROOT.TMath.BreitWigner(4., 3., 3.)
fBW = ROOT.TF1("fBW", "[0]*TMath::BreitWigner(x, [1], [2])", xlow_fit, xup_fit)
setBWLims(fBW)
# declare its own function for signal
fSig = ROOT.TF1("fSig", "[0]*TMath::BreitWigner(x, [1], [2])", xlow_fit, xup_fit)
# first order polynomial as background
fFit1 = ROOT.TF1("fit1", "fBW+pol1(3)", xlow_fit, xup_fit)
setBWLims(fFit1)
# second order polynomial as background
fFit2 = ROOT.TF1("fit2", "fBW+pol2(3)", xlow_fit, xup_fit)
# second order polynomial only
fBkg2 = ROOT.TF1("bkg2", "pol2", xlow_fit, xup_fit)
setBWLims(fFit2)
# third order polynomial as background
fFit3 = ROOT.TF1("fit3", "fBW+pol3(3)", xlow_fit, xup_fit)
setBWLims(fFit3)
fFull = ROOT.TF1("fullManual", "[0]*[2]/((x-[1])*(x-[1]) + [2]*[2]/4) + [3]*x + [4]", xlow_fit, xup_fit)
setBWLims(fFull)
# set color
setColor(fFit1); setColor(fFit2); setColor(fFit3);
setColor(fBW, ROOT.kRed)
setColor(fBkg2, ROOT.kBlue)

### Anpassning med Breit-Wigner

In [35]:
#canvas_fit = ROOT.TCanvas("canvas_fit", "", 800, 600) # open a new canvas
# canvas_fit.cd()
#canvas.Clear()
# h_mass_electrons.Draw("e1p")

#canvas.Draw()
#canvas_fit.Draw()
fitresult=h_mass_electrons.Fit("fit2", "S+", "SAME", xlow_fit, xup_fit) # store fit result
# Draw signal and bkg on top, using fit result
fitresult_parameters={}
for i_p,p in enumerate(fitresult.Parameters()):
    fitresult_parameters[i_p]=(p, fitresult.ParError(i_p))
### signal
for ipar in range(3): fSig.SetParameter(ipar, fitresult_parameters[ipar][0])
# fSig.Draw("SAME")

### bkg
for ipar in range(2): fBkg2.SetParameter(ipar, fitresult_parameters[ipar+3][0])
#fBkg2.Draw("SAME")
#ROOT.disableJSVis()
#ROOT.enableJSVis()
canvas.Draw()
#ROOT.disableJSVis()
#canvas.SaveAs("figures/fit.pdf")
#for obj in canvas.GetListOfPrimitives():
#    print(obj)

 FCN=158.147 FROM MIGRAD    STATUS=CONVERGED      82 CALLS          83 TOTAL
                     EDM=2.662e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           8.12972e+03   1.73597e+02   1.15672e-04  -6.95017e-05
   2  p1           8.98723e+01   7.77877e-02   4.28234e-05  -1.06349e-04
   3  p2           5.53945e+00   1.81854e-01   5.85813e-05  -1.56175e-04
   4  p3           6.52715e+01   5.63674e+00   2.74062e-03  -8.12552e-06
   5  p4          -1.34171e+00   1.51971e-01   2.37609e-05   2.38600e-04
   6  p5           6.30960e-03   8.52827e-04   1.91665e-07  -1.74881e-01


In [ ]:
# Draw signal and bkg on top, using fit result
fitresult_parameters={}
for i_p,p in enumerate(fitresult.Parameters()):
    fitresult_parameters[i_p]=(p, fitresult.ParError(i_p))
# signal
fBW.SetParameter(0, fitresult_parameters[0][0])
fBW.SetParameter(1, fitresult_parameters[1][0])
fBW.SetParameter(2, fitresult_parameters[2][0])
fBW.Draw("SAME")
fBW.Print()
print([fBW.GetParameter(i) for i in range(3)])
# bkg
fBkg2.SetParameter(0, fitresult_parameters[0+3][0])
fBkg2.SetParameter(1, fitresult_parameters[1+3][1])
fBkg2.Draw("SAME")

In [ ]:
canvas.Draw()

In [ ]:
### doesn't seem to work
#def getNumpyArrFromBuffer(buffer,size_buffer):
#    ''' get numpy array from buffer '''
#    import numpy
#    buffer.SetSize(size_buffer)
#    return numpy.array(buffer, copy=True)